In [1]:
import pandas as pd
import pickle
from scipy.signal import welch, butter, lfilter
import numpy as np
import pyeeg as pe
import glob
import os
import matplotlib.pyplot as plt
from brainflow.data_filter import DataFilter, WindowOperations

c:\Users\josee\OneDrive\Desktop\Projects\NeuroH\Coding\.venv\Lib\site-packages\brainflow\board_shim.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [44]:
windowsize = 10

In [45]:
path_models = r"C:\Users\josee\OneDrive\Desktop\Projects\NeuroH\Coding\paper_analysis\models"

Val_Pkl_linear = pickle.load(open(os.path.join(path_models, 'reg_val_model2_{}s.pkl'.format(windowsize)), 'rb'))
Aro_Pkl_linear = pickle.load(open(os.path.join(path_models, 'reg_aro_model2_{}s.pkl'.format(windowsize)), 'rb'))
Dom_Pkl_linear = pickle.load(open(os.path.join(path_models, 'reg_dom_model2_{}s.pkl'.format(windowsize)), 'rb'))

Functions

In [46]:
def resampling(signal, input_fs, output_fs):
    ratio = output_fs / input_fs

    # New length of sample
    n = round(len(signal) * ratio)

    # Perform linear interpolation
    resampled_signal = np.interp(
        np.linspace(0.0, 1.0, n, endpoint=False), # Where to interpret
        np.linspace(0.0, 1.0, len(signal), endpoint=False), # Known positions
        signal,
    )
    return resampled_signal

def brainflow_bandpowers(signal, fs, nfft):

    # Frequency band definition
    bands = [.5,4,8,12,30,45] 

    # Compute the PSD for each frequency band
    psd  = DataFilter.get_psd_welch(signal, nfft=nfft, overlap=nfft//2,sampling_rate=fs, window=WindowOperations.BLACKMAN_HARRIS)

    band_powers = []
    for i in range(len(bands)-1):
        low, high = bands[i], bands[i+1]
        power = DataFilter.get_band_power(psd, low, high)
        band_powers.append(power)

    return band_powers

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y

def compute_psd_bands(data, fs):

    # Define the frequency ranges for each band
    bands = {
        'Delta': (0.5, 4),
        'Theta': (4, 8),
        'Alpha': (8, 12),
        'Beta': (12, 30),
        'Gamma': (30, 45)
    }
    
    # Compute the PSD for each frequency band
    psd_bands = {}
    for band, (f_min, f_max) in bands.items():
        power = pe.bin_power(data, [f_min, f_max], fs)
        psd_bands[band]=np.mean(power)
    
    return psd_bands

#def commavref(data):
    #return referencechannel

def categorize(value):
    if value >= 1 and value < 3:
        category = -1
    elif value >= 3 and value < 6:
        category = 0
    else:
        category = 1

    return category
    

In [47]:
## Emotion and Fear cube ##

emotions = {
    "[-1, -1, -1]": "Sadness",
    "[-1, -1, 0]": "Sadness",
    "[-1, -1, 1]": "Desire",
    "[-1, 0, -1]": "Sadness",
    "[-1, 0, 0]": "Sadness",
    "[-1, 0, 1]": "Desire",
    "[-1, 1, -1]": "Sadness",
    "[-1, 1, 0]": "Sadness",
    "[-1, 1, 1]": "Desire",
    "[0, -1, -1]": "Hate",
    "[0, -1, 0]": "Love",
    "[0, -1, 1]": "Love",
    "[0, 0, -1]": "Sadness",
    "[0, 0, 0]": "Admiration",
    "[0, 0, 1]": "Desire",
    "[0, 1, -1]": "Sadness",
    "[0, 1, 0]": "Admiration",
    "[0, 1, 1]": "Joy",
    "[1, -1, -1]": "Hate",
    "[1, -1, 0]": "Admiration",
    "[1, -1, 1]": "Love",
    "[1, 0, -1]": "Hate",
    "[1, 0, 0]": "Admiration",
    "[1, 0, 1]": "Joy",
    "[1, 1, -1]": "Admiration",
    "[1, 1, 0]": "Admiration",
    "[1, 1, 1]": "Joy"
}

fear_emotions = {
    "[-1, -1, -1]": "Medium Fear",
    "[-1, -1, 0]": "Medium Fear",
    "[-1, -1, 1]": "Low Fear",
    "[-1, 0, -1]": "Medium Fear",
    "[-1, 0, 0]": "Low Fear",
    "[-1, 0, 1]": "No Fear",
    "[-1, 1, -1]": "Low Fear",
    "[-1, 1, 0]": "No Fear",
    "[-1, 1, 1]": "No Fear",
    "[0, -1, -1]": "High Fear",
    "[0, -1, 0]": "Medium Fear",
    "[0, -1, 1]": "Medium Fear",
    "[0, 0, -1]": "Medium Fear",
    "[0, 0, 0]": "Medium Fear",
    "[0, 0, 1]": "Low Fear",
    "[0, 1, -1]": "Medium Fear",
    "[0, 1, 0]": "Low Fear",
    "[0, 1, 1]": "No Fear",
    "[1, -1, -1]": "High Fear",
    "[1, -1, 0]": "High Fear",
    "[1, -1, 1]": "Medium Fear",
    "[1, 0, -1]": "High Fear",
    "[1, 0, 0]": "Medium Fear",
    "[1, 0, 1]": "Low Fear",
    "[1, 1, -1]": "Medium Fear",
    "[1, 1, 0]": "Low Fear",
    "[1, 1, 1]": "Low Fear"
}

def emocion(values):
    arousal_1 = values[0]
    dominance_1 = values[1]
    valence_1 = values[2]
    #valence_1 = int(valence - 1)
    #arousal_1 = int(arousal - 1)
    #dominance_1 = int(dominance - 1)

    # Create the key and ensure it's formatted as a string
    key = str([arousal_1, dominance_1, valence_1])
    emotion_value = emotions.get(key, 'Unknown')
    fear_value = fear_emotions.get(key, 'Unknown')
    return emotion_value, fear_value


In [48]:
## Fear sphere ##
def map_value(val):
    return (val - 1) / (4) - 1

def get_fear_sphere(value):
     # Define angles for rotation in radians
    theta = 2.356194490192345  # Rotation angle around the z-axis
    phi = -0.9553166181245093  # Rotation angle around the x-axis

    r= np.array([
        [np.cos(theta) * np.cos(phi), -np.sin(theta) * np.cos(phi), np.sin(phi)],
        [np.sin(theta), np.cos(theta), 0],
        [-np.sin(phi) * np.cos(theta), np.sin(phi) * np.sin(theta), np.cos(phi)]
    ])


    # Apply the rotation matrix to the mapped vector
    # Map or adjust the input value to [-1,1].
    n_vec = np.dot(r, map_value(value))

    # Decompose the rotated vector into components
    valence, dominance, arousal = n_vec

    # Compute the polar angle phi (angle from the z-axis)
    phi = np.arctan2(np.sqrt(valence ** 2 + dominance ** 2), arousal)

    # Compute the azimuthal angle theta (angle in the xy-plane)
    theta = np.arctan2(valence, dominance)

    fear_metric= np.abs(phi - np.pi) / np.pi, theta

    return fear_metric

def classify_fearm_metric(fear_metric):
    bins= [0, 0.25, 0.50, 0.75, 1.0]
    labels = ["No Fear", "Low Fear", "Medium Fear", "High Fear"]
    index = np.digitize(fear_metric, bins, right=True) - 1
    return labels[index]

In [49]:
## Emotion sphere ##
Descartes_Passions = {
    'Desire': np.array([1,0,0]),
    'Admiration': np.array([0,0,1]),
    'Joy': np.array([1,0,1]),
    'Love': np.array([1,-1,1]),
    'Hate': np.array([-1,-1,1]),
    'Sadness': np.array([-1, 0, -1])
}
DP_NORM = {emotion: vec / np.linalg.norm(vec) for emotion, vec in Descartes_Passions.items()}

def get_emotion_sphere(value):
    """
    Identify the closest matching emotion vector to the given input.

    Input:
    - value (np.array): The input vector representing a detected value for emotion analysis.
        where
        value[0]: valence [1,9], 
        value[1]: dominance [1,9],
        value[2]: arousal [1,9]

    Returns:
    - final_emotion (str): The emotion label with the highest similarity to the normalized input.
    - temp (float): The highest similarity score found.
    - random_emotion (np.array): The normalized version of the input vector on the unit sphere.
    """

    # Map or adjust the input value to [-1,1].
    detected_value = map_value(value)

    # Normalize the mapped detected_value vector to unit length to ensure it lies on the unit sphere.
    random_emotion = detected_value / np.linalg.norm(detected_value)

    # Initialize a temporary variable to store the highest similarity score found.
    temp = 0
    
    # Iterate over each emotion and its normalized vector in DP_NORM.
    for emotion, vec in DP_NORM.items():
        
        # Calculate the cosine similarity between random_emotion and the current emotion vector.
        # Adding 1 and dividing by 2 scales the similarity from [-1, 1] to [0, 1].
        dot = (np.dot(random_emotion, vec) + 1) / 2
        
        # Update temp and final_emotion if the current similarity score is the highest encountered.
        if dot > temp:
            temp, final_emotion = dot, emotion

    # Return the final emotion label with the highest similarity score, the score itself,
    # and the normalized random_emotion vector.
    return final_emotion, temp, random_emotion

Data import

In [50]:
folderpath = r'C:\Users\josee\OneDrive\Desktop\Projects\NeuroH\Coding\Fear_analysis\Data'

pattern = os.path.join(folderpath, r'S{}\EEG\S{:03d}R004*\S{:03d}R004_Complete.csv'.format(1, 1, 1))
matched_files = glob.glob(pattern)
prueba = pd.read_csv(matched_files[0])

In [51]:
subjects = [1,2,3,4,6,8,9,10]
def dataimport(subjects, folderpath, scene):

    rawdata_list = []
    for subject in subjects:
        if subject == 10:
            pattern = os.path.join(folderpath, r'S10\EEG\S10R00{}*\S10R00{}_Complete.csv'.format(scene,scene))
        else:
            pattern = os.path.join(folderpath, r'S{}\EEG\S{:03d}R00{}*\S{:03d}R00{}_Complete.csv'.format(subject, subject, scene, subject,scene))
        
    
        matched_files = glob.glob(pattern)
        if len(matched_files) == 1:    
            holder = pd.read_csv(matched_files[0])[['Timestamp','Channel_1', 'Channel_2', 'Channel_3', 'Channel_4', 'Channel_5', 'Channel_6', 'Channel_7', 'Channel_8']].dropna()
            holder.insert(loc=0, column='Subject', value=[subject] * len(holder))
        else:
            subholder_list = []
            for file in matched_files:
                subholder = pd.read_csv(file)[['Timestamp','Channel_1', 'Channel_2', 'Channel_3', 'Channel_4', 'Channel_5', 'Channel_6', 'Channel_7', 'Channel_8']].dropna()
                subholder_list.append(subholder)
            holder = pd.concat(subholder_list, ignore_index=True)
            holder.insert(loc=0, column='Subject', value=[subject] * len(holder))

        rawdata_list.append(holder)

    rawdata_df = pd.concat(rawdata_list, ignore_index=True)

    return rawdata_df

basaldata = dataimport(subjects, folderpath, scene=1)
feardata = dataimport(subjects, folderpath, scene=4)

In [52]:
feardata[feardata['Subject'] == 1]

,Subject,Timestamp,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8
0,1,1.714396e+09,54197.593534,33385.392533,-18995.205342,-34792.636012,28773.020658,-5365.670367,9815.075673,25402.668166
1,1,1.714396e+09,54084.426652,33276.852462,-18444.190138,-34821.492114,30453.424806,-5470.634159,10674.477896,26907.834638
2,1,1.714396e+09,54073.429593,33450.749034,-19507.730843,-35270.963343,29305.752135,-5547.099477,9688.609503,26102.747155
3,1,1.714396e+09,54175.398251,33577.282259,-20188.475572,-35302.121675,27517.098488,-5451.702231,8722.678867,24547.266906
4,1,1.714396e+09,54215.363171,33431.459478,-19221.472051,-34865.748568,28393.108057,-5369.671329,9558.969386,25127.898172
...,...,...,...,...,...,...,...,...,...,...
42511,1,1.714396e+09,43556.799717,29508.080126,-21632.420615,-36545.415109,26418.554952,-4902.497518,7399.656761,22227.804032
42512,1,1.714396e+09,43551.234132,29601.823342,-22348.414045,-36969.360646,25823.104480,-4984.215496,6829.396704,21879.630909
42513,1,1.714396e+09,43634.337918,29708.016480,-23027.258876,-37117.552712,24182.374678,-4916.467359,5918.093731,20542.773073
42514,1,1.714396e+09,43641.132848,29628.645435,-22437.105767,-36717.791762,24469.169911,-4822.925308,6315.887727,20679.453990


In [53]:
lowcut = 4.0  # Lower cutoff frequency in Hz
highcut = 45  # Upper cutoff frequency in Hz
fs = 250  # Sampling rate in Hz
test = (
    feardata[feardata["Subject"] == 1].copy().drop(["Subject", "Timestamp"], axis=1)
)
filtered_df = test.apply(lambda col: butter_bandpass_filter(col, lowcut, highcut, fs))
average_reference = filtered_df.mean(axis=1)
df_average_reference = filtered_df.sub(average_reference, axis=0)[0:1251]

psd_df = pd.DataFrame()

for column in df_average_reference.columns:
    # Compute the PSD for the column data and frequency bands
    psd_bands = compute_psd_bands(df_average_reference[column].values, fs=fs)

    # Add the PSD values to the DataFrame
    psd_df = pd.concat([psd_df, pd.DataFrame([psd_bands])], ignore_index=True)

# for column in df_average_reference.columns:
#    signal = np.ascontiguousarray(df_average_reference[column].values.astype(np.float64))
# Compute the PSD for the column data and frequency bands
#    psd_bands = brainflow_bandpowers(signal=signal, fs=fs, nfft=256)

# Add the PSD values to the DataFrame
#    psd_df = pd.concat([psd_df, pd.DataFrame([psd_bands])], ignore_index=True)
# psd_df.columns = ['Delta','Theta','Alpha','Beta', 'Gamma']
df_t = psd_df.transpose()

df_t.columns = ["Fp1", "Fp2", "C3", "C4", "P7", "P8", "O1", "O2"]

df_t = df_t.reset_index()

# Use the melt function to reshape the DataFrame
melted_df = pd.melt(df_t, id_vars="index", var_name="channel", value_name="value")

# Convert channel numbers to strings
melted_df["channel"] = melted_df["channel"].astype(str)

# Create a new 'channel_band' column by combining 'channel' and 'index' columns
melted_df["channel_band"] = melted_df["channel"] + "_" + melted_df["index"]

# Pivot the DataFrame to get the desired format
new_df = melted_df.pivot(index="index", columns="channel_band", values="value")

series = new_df.stack()

# Convert the Series back to a DataFrame with a single row
filter_df = pd.DataFrame(series)

valo = filter_df[0]
valores = valo.reset_index(drop=True)
df_modelo = pd.DataFrame(valores).transpose()

df_modelo.columns = [
    "Fp1_Delta",
    "Fp1_Theta",
    "Fp1_Alpha",
    "Fp1_Beta",
    "Fp1_Gamma",
    "Fp2_Delta",
    "Fp2_Theta",
    "Fp2_Alpha",
    "Fp2_Beta",
    "Fp2_Gamma",
    "C3_Delta",
    "C3_Theta",
    "C3_Alpha",
    "C3_Beta",
    "C3_Gamma",
    "C4_Delta",
    "C4_Theta",
    "C4_Alpha",
    "C4_Beta",
    "C4_Gamma",
    "P7_Delta",
    "P7_Theta",
    "P7_Alpha",
    "P7_Beta",
    "P7_Gamma",
    "P8_Delta",
    "P8_Theta",
    "P8_Alpha",
    "P8_Beta",
    "P8_Gamma",
    "O1_Delta",
    "O1_Theta",
    "O1_Alpha",
    "O1_Beta",
    "O1_Gamma",
    "O2_Delta",
    "O2_Theta",
    "O2_Alpha",
    "O2_Beta",
    "O2_Gamma",
]

df_pred = df_modelo.reset_index(drop=True)

CANALES = ["Fp1", "Fp2", "C3", "C4", "P7", "P8", "O1", "O2"]

for channel in CANALES:
    df_pred[f"{channel}_Engagement"] = df_pred[f"{channel}_Beta"] / (
        df_pred[f"{channel}_Theta"] + df_pred[f"{channel}_Alpha"]
    )

for channel in CANALES:
    df_pred[f"{channel}_Fatigue"] = (
        df_pred[f"{channel}_Alpha"] / df_pred[f"{channel}_Theta"]
    )

for channel in CANALES:
    df_pred[f"{channel}_Excitement"] = (
        df_pred[f"{channel}_Beta"] / df_pred[f"{channel}_Alpha"]
    )

for channel in CANALES:
    df_pred[f"{channel}_Relaxation"] = (
        df_pred[f"{channel}_Theta"] / df_pred[f"{channel}_Delta"]
    )

#df_pred

# psd_bands = df_average_reference.apply(
# lambda col: brainflow_bandpowers(signal=col, fs=fs, nfft=256)
# )
# psd_bands


In [54]:
lowcut = 4.0  # Lower cutoff frequency in Hz
highcut = 45  # Upper cutoff frequency in Hz
fs = 250  # Sampling rate in Hz
wsz = windowsize * fs

#ratio = 128/250
def processing(data, lowcut, highcut, fs, wsz):
    featmat_list = []
    for subject in subjects:
        dataholder = data[data['Subject'] == subject][['Channel_1', 'Channel_2', 'Channel_3', 'Channel_4', 'Channel_5', 'Channel_6', 'Channel_7', 'Channel_8']]
    #df5 = dataholder.apply(lambda col: resampling(col.values, in_fs, out_fs))

    # Apply the bandpass filter to each column
        filtered_df = dataholder.apply(lambda col: butter_bandpass_filter(col, lowcut, highcut, fs))

        average_reference = filtered_df.mean(axis=1)
        df_average_reference = filtered_df.sub(average_reference, axis=0)
    #df_average_reference.insert(loc=0, column='Timestamp', value=rawdata_df[rawdata_df['Subject'] == subject]['Timestamp'])
        t = np.arange(0, len(df_average_reference), 1) * 1/fs
    #df_average_reference.insert(loc=0, column='Time', value=t)

    # Create an empty DataFrame to store the PSD results
        feat_list = []
        num_segments = filtered_df.shape[0] // wsz
        for n in range(num_segments):
        #time = df_average_reference.iloc[n*wsz:(n+1)*wsz, :]['Time'].iloc[0]
            timepoint = t[n*wsz:(n+1)*wsz][0]
            df_temp = df_average_reference.iloc[n*wsz:(n+1)*wsz, :]#.drop('Time', axis=1)
        # Iterate over each column in your DataFrame
            psd_df = pd.DataFrame()
        #for column in df_temp.columns:
            # Compute the PSD for the column data and frequency bands
         #   psd_bands = compute_psd_bands(df_temp[column].values, fs=fs)

            # Add the PSD values to the DataFrame
          #  psd_df = pd.concat([psd_df, pd.DataFrame([psd_bands])], ignore_index=True)

            for column in df_average_reference.columns:
                signal = np.ascontiguousarray(df_temp[column].values.astype(np.float64))
            # Compute the PSD for the column data and frequency bands
                psd_bands = brainflow_bandpowers(signal=signal, fs=fs, nfft=256)

            # Add the PSD values to the DataFrame
                psd_df = pd.concat([psd_df, pd.DataFrame([psd_bands])], ignore_index=True)
            psd_df.columns = ['Delta','Theta','Alpha','Beta', 'Gamma']

            df_t = psd_df.transpose()
            df_t.columns = ['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2']

            df_t = df_t.reset_index()

        # Use the melt function to reshape the DataFrame
            melted_df = pd.melt(df_t, id_vars='index', var_name='channel', value_name='value')

        # Convert channel numbers to strings
            melted_df['channel'] = melted_df['channel'].astype(str)

        # Create a new 'channel_band' column by combining 'channel' and 'index' columns
            melted_df['channel_band'] = melted_df['channel'] + '_' + melted_df['index']

        # Pivot the DataFrame to get the desired format
            new_df = melted_df.pivot(index='index', columns='channel_band', values='value')

            series = new_df.stack()

        # Convert the Series back to a DataFrame with a single row
            filter_df = pd.DataFrame(series)

            valo =filter_df[0]
            valores = valo.reset_index(drop=True)
            df_modelo = pd.DataFrame(valores).transpose()

            df_modelo.columns = ['Fp1_Delta', 'Fp1_Theta', 'Fp1_Alpha','Fp1_Beta','Fp1_Gamma',
                            'Fp2_Delta', 'Fp2_Theta', 'Fp2_Alpha','Fp2_Beta','Fp2_Gamma',
                            'C3_Delta', 'C3_Theta', 'C3_Alpha','C3_Beta','C3_Gamma',
                            'C4_Delta', 'C4_Theta', 'C4_Alpha','C4_Beta','C4_Gamma',
                            'P7_Delta', 'P7_Theta', 'P7_Alpha','P7_Beta','P7_Gamma',
                            'P8_Delta', 'P8_Theta', 'P8_Alpha','P8_Beta','P8_Gamma',
                            'O1_Delta', 'O1_Theta', 'O1_Alpha','O1_Beta','O1_Gamma',
                            'O2_Delta', 'O2_Theta', 'O2_Alpha','O2_Beta','O2_Gamma',]

            df_pred = df_modelo.reset_index(drop=True)

            CANALES = ['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2']

            for channel in CANALES:
                df_pred[f'{channel}_Engagement'] = df_pred[f'{channel}_Beta'] / (df_pred[f'{channel}_Theta'] + df_pred[f'{channel}_Alpha'])

            for channel in CANALES:
                df_pred[f'{channel}_Fatigue'] = df_pred[f'{channel}_Alpha'] / df_pred[f'{channel}_Theta']

            for channel in CANALES:
                df_pred[f'{channel}_Excitement'] = df_pred[f'{channel}_Beta'] / df_pred[f'{channel}_Alpha']

            for channel in CANALES:
                df_pred[f'{channel}_Relaxation'] = df_pred[f'{channel}_Theta'] / df_pred[f'{channel}_Delta']
        
            df_pred.insert(0, 'Time', [timepoint])
            feat_list.append(df_pred)
        feat_df = pd.concat(feat_list, ignore_index=True)
        feat_df.insert(loc=0, column='Subject', value=[subject] * len(feat_df))
        featmat_list.append(feat_df)
    featmat_df = pd.concat(featmat_list, ignore_index=True)
    return featmat_df

basalprocessed = processing(data=basaldata, fs=fs, lowcut=lowcut, highcut=highcut, wsz=wsz)
fearprocessed = processing(data=feardata, fs=fs, lowcut=lowcut, highcut=highcut, wsz=wsz)


In [55]:
fearprocessed[fearprocessed['Subject']==10]

,Subject,Time,Fp1_Delta,Fp1_Theta,Fp1_Alpha,Fp1_Beta,Fp1_Gamma,Fp2_Delta,Fp2_Theta,Fp2_Alpha,...,O1_Excitement,O2_Excitement,Fp1_Relaxation,Fp2_Relaxation,C3_Relaxation,C4_Relaxation,P7_Relaxation,P8_Relaxation,O1_Relaxation,O2_Relaxation
112,10,0.0,5.492267,28.595769,6.962367,7.373868,6.318795,89.396702,3.374127,3.597005,...,0.532024,0.022428,5.206551,0.037743,0.687628,234.544383,16.458705,1.273193,0.263718,0.177379
113,10,10.0,9.319101,10.005928,59.703063,18.849079,7.463287,205.879864,10.949012,12.008234,...,1.213623,0.061023,1.073701,0.053182,0.342629,3.893669,20.517331,2.371404,0.659316,0.786881
114,10,20.0,229.083126,134.257343,125.774210,98.917413,200.386851,5492.961907,133.660694,81.024081,...,0.771966,0.025566,0.586064,0.024333,0.857633,14.288031,27.202521,1.252543,0.541888,2.421997
115,10,30.0,80.187404,67.521636,2316.398347,82.569247,73.932553,1155.362084,75.054129,58.952988,...,0.952366,0.053982,0.842048,0.064962,0.064875,16.438718,15.310715,29.368355,0.655912,0.797668
116,10,40.0,6.430032,12.300649,167.461641,307.320692,138.904763,257.999311,97.837070,11.508561,...,2.573506,0.716429,1.913000,0.379214,1.636417,0.921278,0.691980,15.292234,0.184992,0.470283
117,10,50.0,118.966975,58.989352,382.765968,93.806610,83.237843,3429.109990,105.076485,85.324252,...,0.486413,0.027356,0.495846,0.030642,0.511114,6.557680,67.639481,3.543187,0.986059,0.873188
118,10,60.0,93.767375,79.045372,213.238967,45.370812,58.573697,2346.019589,49.875386,91.378231,...,1.089678,0.023860,0.842994,0.021260,0.702564,2.698975,47.533390,1.661426,1.995959,1.021822
119,10,70.0,3.668330,8.766075,5.985959,5.602962,9.856508,225.753291,9.066883,31.611320,...,2.689805,0.033656,2.389664,0.040163,0.456339,0.594488,31.119589,4.041699,1.624253,2.955309
120,10,80.0,472.619013,865.898062,76.221306,102.053057,331.606655,9403.073063,66.451748,288.088642,...,1.089166,0.014700,1.832127,0.007067,1.270501,1.114393,36.898139,0.140105,2.290713,2.938556
121,10,90.0,69.325786,82.103506,81.669415,82.798573,71.672883,3576.942336,80.725092,85.699891,...,0.891162,0.022196,1.184314,0.022568,0.891939,0.974010,47.391100,1.734049,0.909220,0.879518


In [56]:
# Normalización
channel_cols = [col for col in fearprocessed.columns if col not in ['Subject', 'Time']]
normdata_list = []
for subject in subjects:
    ref = basalprocessed[basalprocessed['Subject']==subject].drop(['Subject','Time'], axis=1).mean(axis=0)
    print(subject, ref[0])
    data = fearprocessed[fearprocessed['Subject'] == subject].copy()
    data[channel_cols] = (data[channel_cols] - ref[channel_cols]) / ref[channel_cols]
    normdata_list.append(data)
normdata_df = pd.concat(normdata_list, ignore_index=True)
normdata_df

1 6339.137664262276
2 1178.1856768590362
3 406.447677829382
4 2.179194025779247
6 177.72580702884616
8 1.5627366029561967
9 63.03497435381478
10 13.608608715001777


,Subject,Time,Fp1_Delta,Fp1_Theta,Fp1_Alpha,Fp1_Beta,Fp1_Gamma,Fp2_Delta,Fp2_Theta,Fp2_Alpha,...,O1_Excitement,O2_Excitement,Fp1_Relaxation,Fp2_Relaxation,C3_Relaxation,C4_Relaxation,P7_Relaxation,P8_Relaxation,O1_Relaxation,O2_Relaxation
0,1,0.0,-0.998499,-0.998904,-0.996209,-0.996469,-0.999920,-0.998295,-0.998327,-0.997747,...,-0.358143,-0.761456,-0.891094,-0.876565,-0.834161,26.074663,9.003610,1.643520,0.187624,-0.995417
1,1,10.0,-0.996102,-0.997248,-0.994972,-0.994686,-0.999767,-0.988206,-0.998824,-0.995655,...,-0.764157,-0.993679,-0.894788,-0.987446,-0.808670,-0.812098,7.816455,1.621433,0.360421,-0.924068
2,1,20.0,-0.993318,-0.996665,-0.993935,-0.995172,-0.999059,-0.962609,-0.994584,-0.996190,...,-0.797568,-0.982438,-0.925601,-0.981769,-0.836666,0.004821,4.956898,1.107779,-0.000527,-0.710628
3,1,30.0,-0.310149,-0.583828,-0.528961,-0.614780,-0.562511,6.713118,0.029529,-0.520492,...,-0.771976,-0.978067,-0.910083,-0.983201,-0.859564,-0.546213,0.563081,0.030315,-0.759059,0.540592
4,1,40.0,-0.963150,-0.969425,-0.963427,-0.967788,-0.996968,-0.725283,-0.945095,-0.961332,...,-0.697701,-0.967122,-0.876335,-0.974847,-0.845157,-0.090043,11.181333,0.253799,-0.545595,-0.869294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,10,140.0,4.373769,8.896351,6.536640,2.591436,7.312532,23.065998,7.514064,2.877494,...,-0.149991,-0.940500,-0.229812,-0.867578,-0.108896,1.433836,2.380190,0.225974,0.230297,0.599457
127,10,150.0,-0.840693,-0.217741,-0.486306,-0.789411,-0.646656,-0.941505,-0.757805,-0.927065,...,4.547613,0.189035,1.053608,0.549786,-0.017119,-0.853814,-0.886669,0.227988,-0.239396,-0.340592
128,10,160.0,-0.145620,2.174438,-0.018760,-0.425232,0.366817,2.132552,0.538345,-0.284942,...,0.612797,-0.921216,0.553876,-0.816184,0.025214,-0.203277,1.678624,-0.190736,-0.059371,0.250294
129,10,170.0,1.122056,29.025787,0.066630,-0.275887,1.959024,1.845296,0.847629,0.085420,...,5.929740,-0.897421,4.917498,-0.756940,-0.014130,0.772083,0.546411,-0.898511,0.350910,1.916438


In [57]:
valen_spheric, arous_spheric, domin_spheric = Val_Pkl_linear.predict(normdata_df.drop(['Subject','Time'], axis=1).values).reshape(-1), Aro_Pkl_linear.predict(normdata_df.drop(['Subject','Time'], axis=1).values).reshape(-1), Dom_Pkl_linear.predict(normdata_df.drop(['Subject','Time'], axis=1).values).reshape(-1)

c:\Users\josee\OneDrive\Desktop\Projects\NeuroH\Coding\.venv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\josee\OneDrive\Desktop\Projects\NeuroH\Coding\.venv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\josee\OneDrive\Desktop\Projects\NeuroH\Coding\.venv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [58]:
valen_categorized = list(map(categorize, valen_spheric))
arous_categorized = list(map(categorize, arous_spheric))
domin_categorized = list(map(categorize, domin_spheric))

In [59]:
adv = np.column_stack((arous_categorized, domin_categorized, valen_categorized))
emotion_cube = [emocion(adv[i,:]) for i in range(adv.shape[0])]

In [60]:
emotion_cube = np.array(emotion_cube)

In [61]:
vda = np.column_stack((valen_spheric, domin_spheric, arous_spheric))
emotion_spheric = [get_emotion_sphere(vda[i,:])[0] for i in range(vda.shape[0])]

In [62]:
detected_values = np.array([valen_spheric, domin_spheric, arous_spheric]).T
results = []
for row in detected_values:
    fear_metric, theta = get_fear_sphere(row)
    fear_label = classify_fearm_metric(fear_metric)
    results.append(fear_label)
fear_label_spheric = np.array(results)

In [63]:
vad_mat_arr = np.column_stack([
    normdata_df['Subject'].values,
    normdata_df['Time'].values,
    valen_spheric,
    arous_spheric,
    domin_spheric,
    emotion_spheric,
    fear_label_spheric,
    emotion_cube
])

vad_mat_df = pd.DataFrame(vad_mat_arr, columns=['Subject', 'Timestamp', 'Valence', 'Arousal', 'Dominance','emo_spheric','fear_spheric','emo_cubic','fear_cubic'])
vad_mat_df

,Subject,Timestamp,Valence,Arousal,Dominance,emo_spheric,fear_spheric,emo_cubic,fear_cubic
0,1,0.0,4.8079,4.336100000000001,5.1008,Sadness,Low Fear,Admiration,Medium Fear
1,1,10.0,4.690999999999999,4.3087,5.500100000000001,Sadness,Low Fear,Admiration,Medium Fear
2,1,20.0,4.6796999999999995,4.174699999999999,5.2709,Sadness,Low Fear,Admiration,Medium Fear
3,1,30.0,4.165500000000001,4.735700000000001,6.577700000000002,Sadness,Low Fear,Admiration,Low Fear
4,1,40.0,4.4635,4.378299999999999,5.288599999999999,Sadness,Low Fear,Admiration,Medium Fear
...,...,...,...,...,...,...,...,...,...
126,10,140.0,4.867399999999999,5.135100000000001,5.188299999999999,Admiration,Medium Fear,Admiration,Medium Fear
127,10,150.0,4.747999999999998,5.3298999999999985,4.9574,Hate,High Fear,Admiration,Medium Fear
128,10,160.0,4.4445000000000014,5.151600000000001,5.247000000000001,Sadness,Medium Fear,Admiration,Medium Fear
129,10,170.0,5.160300000000001,5.647,4.7557,Admiration,Medium Fear,Admiration,Medium Fear


In [64]:
normdata_df.to_csv(r'Outputs\featmat_norm_{}s.csv'.format(windowsize), index=False)
vad_mat_df.to_csv(r'Outputs\vadmat_new_{}s.csv'.format(windowsize), index=False)